<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [74]:
""" 
# # Installing Java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Dowload Spark
! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# # Decompressing archives
! tar xf spark-3.2.1-bin-hadoop2.7.tgz
# # installing findspark
! pip install -q findspark
# # installing pyspark
! pip install pyspark==3.2.1
 """

' \n# # Installing Java\n! apt-get install openjdk-8-jdk-headless -qq > /dev/null\n# # Dowload Spark\n! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz\n# # Decompressing archives\n! tar xf spark-3.2.1-bin-hadoop2.7.tgz\n# # installing findspark\n! pip install -q findspark\n# # installing pyspark\n! pip install pyspark==3.2.1\n '

In [75]:
""" 
# Importing os library
import os
# # Java envirorment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # Spark envirorment variable
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
 """

' \n# Importing os library\nimport os\n# # Java envirorment variable\nos.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"\n# # Spark envirorment variable\nos.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"\n '

# Better and consistent library that makes the same function as the code above



In [76]:
! pip install pyspark py4j
! pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**Importing modules and pandas library that will be used later.**

In [77]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import pyodbc
import findspark

findspark.init()

In [78]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

Gooogle Colab Version

In [79]:
""" 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
"""

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [80]:
fille = r"../Final/Parquet/tst/Price.parquet"

stocks = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [81]:
fille = r"../Final/Parquet/tst/AtivosBr.parquet"

data = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [82]:
# Stocks table
stocks.createOrReplaceTempView("General")

data.createOrReplaceTempView("Ativos")

# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

In [83]:
# Already called at Calc Table
Gen = spark.sql("""
SELECT 
    regexp_replace(Ticker, '\.SA$', '') as Ticker,
    TotalAssets,
    TotalDebt,
    date_format(to_date(asOfDate, 'yyyy-MM-dd'), 'dd/MM/yyyy') as Data_de_Referencia,
    EBIT
    FROM General;
""")

In [84]:
Gen.show()

+------+------------+----------+------------------+-----------+
|Ticker| TotalAssets| TotalDebt|Data_de_Referencia|       EBIT|
+------+------------+----------+------------------+-----------+
| BRPR3|  9946283000|1.800798E9|        30/12/2019|  7.64347E8|
| BRPR3| 10654156000|2.265793E9|        30/12/2020|  4.14023E8|
| BRPR3| 11283900000|2.914098E9|        30/12/2021|   2.2584E8|
| BRPR3|  4721464000| 2712000.0|        30/12/2022|-1.787577E9|
|GEOO34|266000000000|   9.41E10|        30/12/2019|    5.376E9|
|GEOO34|253452000000| 7.5066E10|        30/12/2020|     8.47E9|
|GEOO34|198874000000| 3.5185E10|        30/12/2021|   -1.807E9|
|GEOO34|188851000000| 2.6148E10|        30/12/2022|     6.78E8|
| GSHP3|  1699421000|1.411032E9|        30/12/2019|  1.27618E8|
| GSHP3|  1693377000|1.840478E9|        30/12/2020|  -2.4443E8|
| GSHP3|  1726089000|2.084624E9|        30/12/2021|  -6.2605E7|
| GSHP3|  1496797000|1.925892E9|        30/12/2022|   8.6725E7|
|HONB34| 58679000000| 1.6002E10|        

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [91]:
RS = spark.sql("""
SELECT 
    regexp_replace(Ticker, '\.SA$', '') as Ticker,
    Nome_Curto,
    Nome_Longo,
    Capitalizacao_de_Mercado,
    ROUND(Porcentagem_de_Mudanca_do_Mercado_Regular,2 ) AS Porcentagem_de_Mudanca_do_Mercado_Regular,
    ROUND(Mudanca_do_Mercado_Regular,2) AS Mudanca_do_Mercado_Regular,
    date_format(to_date(Horario_do_Mercado_Regular, 'yyyy-MM-dd'), 'dd/MM/yyyy') as Data_de_Referencia,
    Preco_do_Mercado_Regular,
    Maxima_do_Mercado_Regular,
    Minima_do_Mercado_Regular,
    Volume_do_Mercado_Regular,
    Fechamento_Anterior_do_Mercado_Regular,
    Abertura_do_Mercado_Regular
FROM Ativos
""")

AnalysisException: cannot resolve 'Porcentagem_de_Mudanca_do_Mercado_Regular' given input columns: [ativos.Abertura_do_Mercado_Regular, ativos.Capitalizacao_de_Mercado, ativos.Dados_da_Bolsa_Atrasados_em, ativos.Estado_do_Mercado, ativos.Fechamento_Anterior_do_Mercado_Regular, ativos.Horario_do_Mercado_Regular, ativos.Maxima_do_Mercado_Regular, ativos.Minima_do_Mercado_Regular, ativos.Mudanca_do_Mercado_Regular, ativos.Nome_Curto, ativos.Nome_Longo, ativos.Nome_da_Bolsa, ativos.Porcentagem_de_Mudança_do_Mercado_Regular, ativos.Preco_do_Mercado_Regular, ativos.Ticker, ativos.Volume_do_Mercado_Regular, ativos.__index_level_0__]; line 7 pos 10;
'Project [regexp_replace(Ticker#600, .SA$, , 1) AS Ticker#665, Nome_Curto#601, Nome_Longo#602, Capitalizacao_de_Mercado#603L, 'ROUND('Porcentagem_de_Mudanca_do_Mercado_Regular, 2) AS Porcentagem_de_Mudanca_do_Mercado_Regular#666, round(Mudanca_do_Mercado_Regular#589, 2) AS Mudanca_do_Mercado_Regular#667, date_format(cast(to_date(Horario_do_Mercado_

In [ ]:
 Preco_do_Mercado_Regular,
    Maxima_do_Mercado_Regular,
    Minima_do_Mercado_Regular,
    Volume_do_Mercado_Regular,
    Fechamento_Anterior_do_Mercado_Regular,
    Abertura_do_Mercado_Regular

In [57]:
RS.show()

+-----------------------------------------+--------------------------+--------------------------+------------------------+-------------------------+-------------------------+-------------------------+--------------------------------------+---------------------------+-------------+---------------------------+-----------------+------+--------------------+--------------------+------------------------+-----------------+
|Porcentagem_de_Mudança_do_Mercado_Regular|Mudança_do_Mercado_Regular|Horario_do_Mercado_Regular|Preço_do_Mercado_Regular|Maxima_do_Mercado_Regular|Minima_do_Mercado_Regular|Volume_do_Mercado_Regular|Fechamento_Anterior_do_Mercado_Regular|Abertura_do_Mercado_Regular|Nome_da_Bolsa|Dados_da_Bolsa_Atrasados_em|Estado_do_Mercado|Ticker|          Nome_Curto|          Nome_Longo|Capitalizacao_de_Mercado|__index_level_0__|
+-----------------------------------------+--------------------------+--------------------------+------------------------+-------------------------+------------

ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

Pandas setting DataFrame to pandas format:

Excel format tables

In [12]:
df_rs.to_excel(r"../Final/RSCH.xlsx",
                sheet_name="research")

In [13]:
df_rs

,Ticker,Preco_Atual,Fluxo_de_caixa_total,Fluxo_de_caixa_por_acao,EBITDA,Divida_total,Liquidez_imediata,Liquidez_corrente,Receita_total,Divida_Patrimonio,...,Fluxo_de_caixa_operacional,Crescimento_de_Receita_3T,Margem_Bruta,Margem_EBITIDA,Margem_Operacional,Margem_liquida,Moeda,Crescimento_dos_ganhos_3T,__index_level_0__,valor_de_mercado
0,ARML3.SA,13.44,8.593540e+08,2.489,3.809280e+08,1.886783e+09,2.691,2.789,838937024,165.614,...,-675387008,1.522,0.45842,0.45406,0.33634,0.14750,BRL,1.841,7,4650455040
1,TTEN3.SA,12.57,8.791550e+08,1.769,5.706630e+08,1.119887e+09,0.777,1.554,6885838848,39.733,...,257696992,0.319,0.15151,0.08287,0.07808,0.08299,BRL,0.342,10,6247516160
2,VITT3.SA,11.19,7.697600e+07,0.541,1.896870e+08,2.642490e+08,1.318,2.058,892286016,47.096,...,130406000,0.085,0.37012,0.21259,0.20659,0.15913,BRL,0.270,15,1594519040
3,CXSE3.SA,8.54,1.272550e+09,0.424,2.892785e+09,0.000000e+00,9.218,9.229,3409924096,NaN,...,2275187968,0.595,0.92936,0.84834,0.84834,0.74776,BRL,0.557,18,25620000768
4,GGPS3.SA,11.84,1.295590e+09,1.935,9.647340e+08,1.937756e+09,1.763,1.895,8658241536,77.118,...,278648992,0.463,0.16126,0.11142,0.09313,0.05279,BRL,0.159,21,7942899712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,SAPR3.SA,3.25,1.207932e+09,0.799,2.242618e+09,5.083244e+09,1.444,1.516,5673647104,57.850,...,1991254016,0.067,0.57992,0.39527,0.33128,0.20296,BRL,0.057,357,5098242560
92,CGAS5.SA,129.01,2.155312e+09,16.264,3.224497e+09,7.424176e+09,1.164,1.251,17414152192,718.872,...,3439486976,0.272,0.19823,0.18517,0.15654,0.10402,BRL,0.238,362,16325827584
93,CGAS3.SA,125.00,2.155312e+09,16.264,3.224497e+09,7.424176e+09,1.164,1.251,17414152192,718.872,...,3439486976,0.272,0.19823,0.18517,0.15654,0.10402,BRL,0.238,363,16783624192
94,EGIE3.SA,39.10,2.445815e+09,2.998,6.159306e+09,1.815048e+10,0.735,1.253,11906605056,215.049,...,3337492992,0.120,0.46942,0.51730,0.43491,0.22366,BRL,10.496,368,32173551616
